In [3]:
import os

import pymongo
from bson.json_util import dumps, loads
from bson.objectid import ObjectId

from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.common.exceptions import TimeoutException

import pandas as pd

import re
from pprint import pprint
from datetime import datetime as dt
import time
import isodate

import sys
# sys.path.append('C:\\Users\\soria\\Documents\\WashUDataDocuments\\HwActivites\\YouTube_Project\\')

import pandas as pd
import json
import os
from pprint import pprint
import random

In [279]:
random.uniform(0, 5)

4.74672464601441

In [20]:
executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
# browser = Browser('chrome', **executable_path)
driver = webdriver.Chrome()

site map, grab: topics/cuisine, topics/meal, topics/method, topics/ingredient/eggs, grains, meats-and-poultry, pasta, seafood, vegetables, 

In [27]:
url = 'https://www.seriouseats.com/sitemap'
driver.get(url)

In [28]:

html = driver.page_source
se_sm_soup = bs(html, 'html.parser')


In [29]:
def topic_anchor(href):
    return re.compile('topics').search(href)
# topic_anchs = list(anchors.filter(lambda x: re.match(r'topics', x['href'])))

In [30]:
se_recipe_links_dict = {}
topic_anchors = se_sm_soup.find_all('a', href=re.compile('topics'))
topic_anchors

[<a data-click-category="nav" data-click-id="recipes-chicken" href="https://www.seriouseats.com/recipes/topics/ingredient/meats-and-poultry/chicken">
 <div class="icon-wrapper">
 <span class="icon icon-food-chicken-white"></span>
 </div>
 <span class="label label-topic">Chicken</span>
 </a>,
 <a data-click-category="nav" data-click-id="recipes-pizza" href="https://www.seriouseats.com/recipes/topics/meal/pizza">
 <div class="icon-wrapper">
 <span class="icon icon-food-pizza-white"></span>
 </div>
 <span class="label label-topic">Pizza</span>
 </a>,
 <a data-click-category="nav" data-click-id="recipes-cocktails" href="https://www.seriouseats.com/recipes/topics/meal/drinks/cocktails">
 <div class="icon-wrapper">
 <span class="icon icon-food-cocktail-white"></span>
 </div>
 <span class="label label-topic">Cocktails</span>
 </a>,
 <a data-click-category="nav" data-click-id="recipes-pasta" href="https://www.seriouseats.com/recipes/topics/ingredient/pasta">
 <div class="icon-wrapper">
 <span 

In [31]:

se_recipe_links_dict['ingredient'] = []
se_recipe_links_dict['method'] = []
se_recipe_links_dict['meal'] = []
se_recipe_links_dict['cuisine'] = []

for i, anchor in enumerate(topic_anchors):
#     print(f"{i}: {anchor.text.strip()}")
#     print(anchor['href'])
    link = anchor['href']
    if 'ingredient' in link:
        this_dict = {'topic': anchor.text.strip(), 'ingredient_url(s)': [link]}
        se_recipe_links_dict['ingredient'].append(this_dict)
    if 'method' in link:
        this_dict = {'topic': anchor.text.strip(), 'method_url(s)': [link]}
        se_recipe_links_dict['method'].append(this_dict)
    if 'meal' in link:
        this_dict = {'topic': anchor.text.strip(), 'meal_url(s)': [link]}
        se_recipe_links_dict['meal'].append(this_dict)
    if 'cuisine' in link:
        this_dict = {'topic': anchor.text.strip(), 'cuisine_url(s)': [link]}
        se_recipe_links_dict['cuisine'].append(this_dict)


In [34]:
se_recipe_links_dict['cuisine'] = se_recipe_links_dict['cuisine'][1:]
se_recipe_links_dict['ingredient'] = se_recipe_links_dict['ingredient'][0:2] + se_recipe_links_dict['ingredient'][3:]
se_recipe_links_dict['method'] = se_recipe_links_dict['method'][0:1] + se_recipe_links_dict['method'][2:]
pprint(se_recipe_links_dict)

{'cuisine': [{'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/american'],
              'topic': 'American Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/asian'],
              'topic': 'Asian Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/caribbean'],
              'topic': 'Caribbean Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/asian/chinese'],
              'topic': 'Chinese Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/asian/filipino'],
              'topic': 'Filipino Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/french'],
              'topic': 'French Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/indian'],
              'topic': 'Indian Recipes'},
             {'

In [23]:
def href_filt(href):
    return re.compile('recipe').search(href) and re.compile('https').search(href) and not re.compile('topics').search(href)

In [ ]:
cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\{channel}_youtube_scrape_html.txt"    if os.path.isfile(cacheFilePath):
        with open(cacheFilePath, encoding='utf-8') as cacheFile:
            html = cacheFile.read() 
            
            html = driver.page_source
        with open(cacheFilePath, "w", encoding='utf-8') as cacheFile:
            cacheFile.write(html)

        video_soup = bs(html, 'html.parser')

In [180]:
test = {'cuisine': se_recipe_links_dict['cuisine']}
for topic, _list in se_recipe_links_dict.items():
    for _dict in _list:
        string = topic + '_url(s)'
        url = _dict[string][0]
        driver.get(url)
        html = driver.page_source
        se_recipe_soup = bs(html, 'html.parser')
        content = se_recipe_soup.find('div', class_='content-main')
        recipe_anchors = content.find_all('a', href=href_filt)
        _dict['recipes'] = []
        recipe_anchors = recipe_anchors[2:]
        for anchor in recipe_anchors[::2]:
            this_dict = {'recipe_title': re.sub(r'\n', '', anchor.text), 'recipe_url': anchor['href']}
            _dict['recipes'].append(this_dict)
        while se_recipe_soup.find('a', text=(re.compile('Next'))):
            url = se_recipe_soup.find('a', text=(re.compile('Next')))['href']
            _dict['cuisine_url(s)'].append(url)
            driver.get(url)
            html = driver.page_source
            se_recipe_soup = bs(html, 'html.parser')
            content = se_recipe_soup.find('div', class_='content-main')
            recipe_anchors = content.find_all('a', href=href_filt)
            for anchor in recipe_anchors[::2]:
                this_dict = {'recipe_title': re.sub(r'\n', '', anchor.text), 'recipe_url': anchor['href']}
                _dict['recipes'].append(this_dict)

In [232]:
content = se_recipe_soup.find('div', class_='content-main')
recipe_anchors = content.find_all('a', href=href_filt)
for anchor in recipe_anchors[::2]:
    print(re.sub(r'\n', '', anchor.text))

          Recipes
Grilled CubanosDouble dose of pig.
Baby Back Ribs With Mojo Barbecue SauceGet a little Cuban mojo.
Cook the Book: Jerk Pork
Snapper With Brown Rice, Avocado, and Cheese
Spicy Cuban Shrimp Soup With Noodles
Cuban Beef Casserole with Black Beans and Cumin Rice
Grilled Cuban Sandwich
Dinner Tonight: Cuban Sandwich
Crispy, Shredded Cuban Beef With Pan-Fried Onions and Plantains


In [ ]:
for i in range(len(_dict['cuisine_url(s)'])):
            cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\{_dict['topic']}_{i}.txt"

In [33]:
cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches"
filess = []
for root, dirs, files in os.walk(cacheFilePath):

    for file in files:
        filess.append(file)


['African_Recipes_0.txt',
 'African_Recipes_1.txt',
 'American_Recipes_0.txt',
 'American_Recipes_1.txt',
 'American_Recipes_2.txt',
 'Asian_Recipes_0.txt',
 'Asian_Recipes_1.txt',
 'Asian_Recipes_10.txt',
 'Asian_Recipes_11.txt',
 'Asian_Recipes_12.txt',
 'Asian_Recipes_13.txt',
 'Asian_Recipes_14.txt',
 'Asian_Recipes_15.txt',
 'Asian_Recipes_16.txt',
 'Asian_Recipes_17.txt',
 'Asian_Recipes_18.txt',
 'Asian_Recipes_19.txt',
 'Asian_Recipes_2.txt',
 'Asian_Recipes_20.txt',
 'Asian_Recipes_21.txt',
 'Asian_Recipes_22.txt',
 'Asian_Recipes_23.txt',
 'Asian_Recipes_24.txt',
 'Asian_Recipes_25.txt',
 'Asian_Recipes_26.txt',
 'Asian_Recipes_27.txt',
 'Asian_Recipes_28.txt',
 'Asian_Recipes_29.txt',
 'Asian_Recipes_3.txt',
 'Asian_Recipes_30.txt',
 'Asian_Recipes_31.txt',
 'Asian_Recipes_32.txt',
 'Asian_Recipes_33.txt',
 'Asian_Recipes_4.txt',
 'Asian_Recipes_5.txt',
 'Asian_Recipes_6.txt',
 'Asian_Recipes_7.txt',
 'Asian_Recipes_8.txt',
 'Asian_Recipes_9.txt',
 'Asparagus_Recipes_0.txt',

In [41]:
for i, file in enumerate(filess):
    print(f"processing:{i} of {len(filess)}: {file}")
    for topic, _list in se_recipe_links_dict.items():
        for _dict in _list:
            _dict.setdefault('recipes', [])
            cfTopic = re.sub(r'\s', '_', _dict['topic'])
            cfTopic = re.sub(r'/', '_', cfTopic)
            if cfTopic in file:
                cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\{file}"
                with open(cacheFilePath, encoding='utf-8') as cacheFile:
                    html = cacheFile.read()
                    se_recipe_soup = bs(html, 'html.parser')
                    content = se_recipe_soup.find('div', class_='content-main')
                    recipe_anchors = content.find_all('a', href=href_filt)
                    recipe_anchors = recipe_anchors[2:]
                    for anchor in recipe_anchors[::2]:
                        this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                        _dict['recipes'].append(this_dict)

processing:0 of 1310: African_Recipes_0.txt
processing:1 of 1310: African_Recipes_1.txt
processing:2 of 1310: American_Recipes_0.txt
processing:3 of 1310: American_Recipes_1.txt
processing:4 of 1310: American_Recipes_2.txt
processing:5 of 1310: Asian_Recipes_0.txt
processing:6 of 1310: Asian_Recipes_1.txt
processing:7 of 1310: Asian_Recipes_10.txt
processing:8 of 1310: Asian_Recipes_11.txt
processing:9 of 1310: Asian_Recipes_12.txt
processing:10 of 1310: Asian_Recipes_13.txt
processing:11 of 1310: Asian_Recipes_14.txt
processing:12 of 1310: Asian_Recipes_15.txt
processing:13 of 1310: Asian_Recipes_16.txt
processing:14 of 1310: Asian_Recipes_17.txt
processing:15 of 1310: Asian_Recipes_18.txt
processing:16 of 1310: Asian_Recipes_19.txt
processing:17 of 1310: Asian_Recipes_2.txt
processing:18 of 1310: Asian_Recipes_20.txt
processing:19 of 1310: Asian_Recipes_21.txt
processing:20 of 1310: Asian_Recipes_22.txt
processing:21 of 1310: Asian_Recipes_23.txt
processing:22 of 1310: Asian_Recipes_

processing:187 of 1310: Clam,_Mussel,_and_Oyster_Recipes_4.txt
processing:188 of 1310: Cocktails_0.txt
processing:189 of 1310: Cocktails_1.txt
processing:190 of 1310: Cocktails_10.txt
processing:191 of 1310: Cocktails_11.txt
processing:192 of 1310: Cocktails_12.txt
processing:193 of 1310: Cocktails_13.txt
processing:194 of 1310: Cocktails_14.txt
processing:195 of 1310: Cocktails_15.txt
processing:196 of 1310: Cocktails_16.txt
processing:197 of 1310: Cocktails_17.txt
processing:198 of 1310: Cocktails_18.txt
processing:199 of 1310: Cocktails_19.txt
processing:200 of 1310: Cocktails_2.txt
processing:201 of 1310: Cocktails_20.txt
processing:202 of 1310: Cocktails_21.txt
processing:203 of 1310: Cocktails_22.txt
processing:204 of 1310: Cocktails_23.txt
processing:205 of 1310: Cocktails_24.txt
processing:206 of 1310: Cocktails_25.txt
processing:207 of 1310: Cocktails_26.txt
processing:208 of 1310: Cocktails_27.txt
processing:209 of 1310: Cocktails_28.txt
processing:210 of 1310: Cocktails_29.t

processing:389 of 1310: Eggs_16.txt
processing:390 of 1310: Eggs_17.txt
processing:391 of 1310: Eggs_18.txt
processing:392 of 1310: Eggs_19.txt
processing:393 of 1310: Eggs_2.txt
processing:394 of 1310: Eggs_20.txt
processing:395 of 1310: Eggs_3.txt
processing:396 of 1310: Eggs_4.txt
processing:397 of 1310: Eggs_5.txt
processing:398 of 1310: Eggs_6.txt
processing:399 of 1310: Eggs_7.txt
processing:400 of 1310: Eggs_8.txt
processing:401 of 1310: Eggs_9.txt
processing:402 of 1310: Eggs_Benedict_Recipes_0.txt
processing:403 of 1310: Egg_Salad_Recipes_0.txt
processing:404 of 1310: Filipino_Recipes_0.txt
processing:405 of 1310: Fish_Recipes_0.txt
processing:406 of 1310: Fish_Recipes_1.txt
processing:407 of 1310: Fish_Recipes_10.txt
processing:408 of 1310: Fish_Recipes_11.txt
processing:409 of 1310: Fish_Recipes_12.txt
processing:410 of 1310: Fish_Recipes_13.txt
processing:411 of 1310: Fish_Recipes_14.txt
processing:412 of 1310: Fish_Recipes_15.txt
processing:413 of 1310: Fish_Recipes_16.txt

processing:571 of 1310: Meats_and_Poultry_23.txt
processing:572 of 1310: Meats_and_Poultry_24.txt
processing:573 of 1310: Meats_and_Poultry_25.txt
processing:574 of 1310: Meats_and_Poultry_26.txt
processing:575 of 1310: Meats_and_Poultry_27.txt
processing:576 of 1310: Meats_and_Poultry_28.txt
processing:577 of 1310: Meats_and_Poultry_29.txt
processing:578 of 1310: Meats_and_Poultry_3.txt
processing:579 of 1310: Meats_and_Poultry_30.txt
processing:580 of 1310: Meats_and_Poultry_31.txt
processing:581 of 1310: Meats_and_Poultry_32.txt
processing:582 of 1310: Meats_and_Poultry_33.txt
processing:583 of 1310: Meats_and_Poultry_34.txt
processing:584 of 1310: Meats_and_Poultry_35.txt
processing:585 of 1310: Meats_and_Poultry_36.txt
processing:586 of 1310: Meats_and_Poultry_37.txt
processing:587 of 1310: Meats_and_Poultry_38.txt
processing:588 of 1310: Meats_and_Poultry_39.txt
processing:589 of 1310: Meats_and_Poultry_4.txt
processing:590 of 1310: Meats_and_Poultry_40.txt
processing:591 of 1310

processing:750 of 1310: Pork_Recipes_17.txt
processing:751 of 1310: Pork_Recipes_18.txt
processing:752 of 1310: Pork_Recipes_19.txt
processing:753 of 1310: Pork_Recipes_2.txt
processing:754 of 1310: Pork_Recipes_20.txt
processing:755 of 1310: Pork_Recipes_21.txt
processing:756 of 1310: Pork_Recipes_22.txt
processing:757 of 1310: Pork_Recipes_23.txt
processing:758 of 1310: Pork_Recipes_24.txt
processing:759 of 1310: Pork_Recipes_25.txt
processing:760 of 1310: Pork_Recipes_26.txt
processing:761 of 1310: Pork_Recipes_27.txt
processing:762 of 1310: Pork_Recipes_28.txt
processing:763 of 1310: Pork_Recipes_29.txt
processing:764 of 1310: Pork_Recipes_3.txt
processing:765 of 1310: Pork_Recipes_30.txt
processing:766 of 1310: Pork_Recipes_31.txt
processing:767 of 1310: Pork_Recipes_32.txt
processing:768 of 1310: Pork_Recipes_4.txt
processing:769 of 1310: Pork_Recipes_5.txt
processing:770 of 1310: Pork_Recipes_6.txt
processing:771 of 1310: Pork_Recipes_7.txt
processing:772 of 1310: Pork_Recipes_8

processing:943 of 1310: Stovetop_Recipes_2.txt
processing:944 of 1310: Stovetop_Recipes_3.txt
processing:945 of 1310: Stovetop_Recipes_4.txt
processing:946 of 1310: Stovetop_Recipes_5.txt
processing:947 of 1310: Stovetop_Recipes_6.txt
processing:948 of 1310: Stovetop_Recipes_7.txt
processing:949 of 1310: Stovetop_Recipes_8.txt
processing:950 of 1310: Stovetop_Recipes_9.txt
processing:951 of 1310: Tequila_Drinks_0.txt
processing:952 of 1310: Tequila_Drinks_1.txt
processing:953 of 1310: Tequila_Drinks_2.txt
processing:954 of 1310: Tequila_Drinks_3.txt
processing:955 of 1310: Tequila_Drinks_4.txt
processing:956 of 1310: Tex-Mex_Recipes_0.txt
processing:957 of 1310: Tex-Mex_Recipes_1.txt
processing:958 of 1310: Thai_Recipes_0.txt
processing:959 of 1310: Thai_Recipes_1.txt
processing:960 of 1310: Thai_Recipes_2.txt
processing:961 of 1310: Thai_Recipes_3.txt
processing:962 of 1310: Thai_Recipes_4.txt
processing:963 of 1310: Thai_Recipes_5.txt
processing:964 of 1310: Thai_Recipes_6.txt
proces

In [10]:
test = {'cuisine': se_recipe_links_dict['cuisine']}
for topic, _list in se_recipe_links_dict.items():
    for _dict in _list:
        _dict.setdefault('recipes', [])
        string = topic + '_url(s)'
        print(f"string: {string}")
        for i, u in enumerate(_dict[string]):
            cfTopic = re.sub(r'\s', '_', _dict['topic'])
            cfTopic = re.sub(r'/', '_', cfTopic)
            print(cfTopic)
            cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\recipe_urls\\{cfTopic}_{i}.txt"
            print(f"cfp: {cacheFilePath}")
            if os.path.isfile(cacheFilePath):
                with open(cacheFilePath, encoding='utf-8') as cacheFile:
                    html = cacheFile.read()
                    se_recipe_soup = bs(html, 'html.parser')
                    content = se_recipe_soup.find('div', class_='content-main')
                    recipe_anchors = content.find_all('a', href=href_filt)
                    for anchor in recipe_anchors[::2]:
                        this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                        _dict['recipes'].append(this_dict)
            else:
                url = u
                print(f"url: {url}")
                time.sleep(random.uniform(0,3))
                driver.get(url)
                html = driver.page_source
                with open(cacheFilePath, "w", encoding='utf-8') as cacheFile:
                    cacheFile.write(html)        

                se_recipe_soup = bs(html, 'html.parser')
                content = se_recipe_soup.find('div', class_='content-main')
                recipe_anchors = content.find_all('a', href=href_filt)
                recipe_anchors = recipe_anchors[2:]
                for anchor in recipe_anchors[::2]:
                    this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                    _dict['recipes'].append(this_dict)
                if se_recipe_soup.find('a', text=(re.compile('Next'))):
                    time.sleep(random.uniform(0,5))
                    url = se_recipe_soup.find('a', text=(re.compile('Next')))['href']
                    _dict[string].append(url)
                    driver.get(url)
                    html = driver.page_source
                    se_recipe_soup = bs(html, 'html.parser')
                    content = se_recipe_soup.find('div', class_='content-main')
                    recipe_anchors = content.find_all('a', href=href_filt)
                    for anchor in recipe_anchors[::2]:
                        this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                        _dict['recipes'].append(this_dict)

string: ingredient_url(s)
Chicken
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Chicken_0.txt
string: ingredient_url(s)
Pasta
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Pasta_0.txt
string: ingredient_url(s)
Cheese
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Cheese_0.txt
string: ingredient_url(s)
Eggs
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Eggs_0.txt
string: ingredient_url(s)
Deviled_Egg_Recipes
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Deviled_Egg_Recipes_0.txt
string: ingredient_url(s)
Egg_Salad_Recipes
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Egg_Salad_Recipes_0.txt
string: ingredient_url(s)
Eggs_Benedict_Recipes
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Eggs_Benedict_Recipes_0.txt
string: ingredient_url(s)
Fried_Egg_Recipes
cfp: C:\Users\soria\Documents\resources\caches\serious_e

string: method_url(s)
Braising_and_Stewing_Recipes
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Braising_and_Stewing_Recipes_0.txt
string: method_url(s)
Deep_Fried_Recipes
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Deep_Fried_Recipes_0.txt
string: method_url(s)
Grilling,_Smoking,_and_Barbecue_Recipes
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Grilling,_Smoking,_and_Barbecue_Recipes_0.txt
string: method_url(s)
No-Cook_Recipes
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\No-Cook_Recipes_0.txt
string: method_url(s)
Poaching_Recipes
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Poaching_Recipes_0.txt
string: method_url(s)
Pressure_Cooker_Recipes
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\Pressure_Cooker_Recipes_0.txt
string: method_url(s)
Roasting_Recipes
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches

In [ ]:
for topic, _list in se_recipe_links_dict.items():
    print(f'topic: {topic}')
    for _dict in _list:
        string = topic + '_url(s)'
        print(f"    string: {string}")
        for i, u in enumerate(_dict[string]):
            print(f'length: {len(_dict[string])}')
            cfTopic = re.sub(r'\s', '_', _dict['topic'])
            cfTopic = re.sub(r'/', '_', cfTopic)
            print(f'        {cfTopic}')
            cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\{cfTopic}_{i}.txt"
            print(f"            cfp: {cacheFilePath}")

In [4]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.serious_eats_test


In [44]:
se_recipe_links_dict.pop('_id')
for topic, _list in se_recipe_links_dict.items():
    collection = mongoDb[topic]
    collection.insert_many(_list)

BulkWriteError: batch op errors occurred

In [55]:
## use if se_recipe_links_dict is created from scratch

for topic, topic_list in se_recipe_links_dict.items():
    print(f'topic: {topic}')
    for topic_dict in topic_list:
    
        for i, recipe in enumerate(topic_dict['recipes']):
            url = recipe['recipe_url']
            print(f"{topic}: {i} of {len(topic_dict['recipes'])} => {url}")
            cfTitle = re.sub(r"\s|/|\.|,|'", '_', recipe['recipe_title'])
            cfTitle = re.sub(r'__', '_', cfTopic)
            print(f"cfTitle: {cfTitle}")
            
            driver.get(url)
            html = driver.page_source
            
            cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\{cfTitle}.txt"
            print(f"cfp: {cacheFilePath}")
            
            with open(cacheFilePath, encoding='utf-8') as cacheFile:
                cacheFile.write(html)
            se_soup = bs(html, 'html.parser')
            script_divs = se_soup.find_all('script', {'type': 'application/ld+json'})
            for div in script_divs:
                j = json.loads(div.text, strict=False)

                recipe.setdefault('category_trees', [])
                recipe.setdefault('recipe', {})
                
                if j['@type'] == 'BreadcrumbList':
                    category_tree = []
                    for item in j['itemListElement']:
                        category_tree.append(item['item']['name'])
                    recipe['category_trees'].append(category_tree)
                if j['@type'] == 'Recipe':
                    try:
                        rating_count = j['aggregateRating']['ratingCount']
                    except:
                        rating_count = False
                    try:
                        rating_value = j['aggregateRating']['ratingValue']
                    except:
                        rating_value = False
                    try:
                        author = j['author']['name']
                    except:
                        author = False
                    try:
                        job_title = j['author']['jobTitle']
                    except:
                        job_title = False
                    try:
                        description = j['description']
                    except:
                        description = False
                    try:
                        keywords = j['keywords']
                    except:
                        keywords = False
                    try: 
                        name = j['name']
                    except:
                        name = False
                    try:
                        categories = j['recipeCategory']
                    except:
                        categories = False
                    try:
                        cuisine = j['recipeCuisine']
                    except:
                        cuisine = False
                    try:
                        ingredients = j['recipeIngredient']
                    except:
                        ingredients = False
                    try:   
                        steps = {}
                        for i, step in enumerate(j['recipeInstructions']):
                            this = 'step_' + str(i+1)
                            steps[this] = step['text']
                    except:
                        steps = False
                    try:
                        recipe_yield = j['recipeYield']
                    except:
                        recipe_yield = False
                    try:
                        time = str(isodate.parse_duration(j['totalTime']).total_seconds()/60)
                    except:
                        time = False
                    recipe['recipe'] = {
                        'name': name, 'rating_count': rating_count, 'rating_value': rating_value, 'author': author,
                        'job_title': job_title, 'description': description, 'keywords': keywords, 'categories': categories,
                        'cuisine': cuisine, 'ingredients': ingredients, 'steps': steps, 'recipe_yield': recipe_yield, 'time':time
                    }



topic: ingredient
ingredient: 0 of 2168 => https://www.seriouseats.com/recipes/2011/12/serious-eats-halal-cart-style-chicken-and-rice-white-sauce-recipe.html
cfTitle: _30-Minute_Pressure_Cooker_Pho_Ga_(Vietnamese_Chicken_Noodle_Soup)_
cfp: C:\Users\soria\Documents\resources\caches\serious_eats_html_caches\_30-Minute_Pressure_Cooker_Pho_Ga_(Vietnamese_Chicken_Noodle_Soup)_.txt


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\_30-Minute_Pressure_Cooker_Pho_Ga_(Vietnamese_Chicken_Noodle_Soup)_.txt'

In [26]:
html = driver.page_source
soup = bs(html, 'html.parser')
content = soup.find('div', class_='content-main')
recipe_anchors = content.find_all('a', href=href_filt)
recipe_anchors = recipe_anchors[2:]
for anchor in recipe_anchors[:2:2]:
    print(anchor)
#     this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
#     _dict['recipes'].append(this_dict)

<a class="module__link" href="https://www.seriouseats.com/recipes/2011/12/serious-eats-halal-cart-style-chicken-and-rice-white-sauce-recipe.html">
<h4 class="title">Serious Eats' Halal Cart-Style Chicken and Rice With White Sauce </h4>
<p class="kicker">Take the Halal cart home with you.</p>
</a>


In [21]:
## use if pulled from mongo

for topic, topic_dict in se_recipe_links_dict.items():
    print(f'topic: {topic}')
    for i, recipe in enumerate(topic_dict['recipes']):
        url = recipe['recipe_url']
        print(f"{topic}: {i} of {len(topic_dict['recipes'])} => {url}")
        driver.get(url)
        html = driver.page_source
        se_soup = bs(html, 'html.parser')
        script_divs = se_soup.find_all('script', {'type': 'application/ld+json'})
        for div in script_divs:
            j = json.loads(div.text, strict=False)

            recipe.setdefault('category_trees', [])
            recipe.setdefault('recipe', {})

            if j['@type'] == 'BreadcrumbList':
                category_tree = []
                for item in j['itemListElement']:
                    category_tree.append(item['item']['name'])
                recipe['category_trees'].append(category_tree)
            if j['@type'] == 'Recipe':
                try:
                    rating_count = j['aggregateRating']['ratingCount']
                except:
                    rating_count = False
                try:
                    rating_value = j['aggregateRating']['ratingValue']
                except:
                    rating_value = False
                try:
                    author = j['author']['name']
                except:
                    author = False
                try:
                    job_title = j['author']['jobTitle']
                except:
                    job_title = False
                try:
                    description = j['description']
                except:
                    description = False
                try:
                    keywords = j['keywords']
                except:
                    keywords = False
                try: 
                    name = j['name']
                except:
                    name = False
                try:
                    categories = j['recipeCategory']
                except:
                    categories = False
                try:
                    cuisine = j['recipeCuisine']
                except:
                    cuisine = False
                try:
                    ingredients = j['recipeIngredient']
                except:
                    ingredients = False
                try:   
                    steps = {}
                    for i, step in enumerate(j['recipeInstructions']):
                        this = 'step_' + str(i+1)
                        steps[this] = step['text']
                except:
                    steps = False
                try:
                    recipe_yield = j['recipeYield']
                except:
                    recipe_yield = False
                try:
                    time = str(isodate.parse_duration(j['totalTime']).total_seconds()/60)
                except:
                    time = False
                recipe['recipe'] = {
                    'name': name, 'rating_count': rating_count, 'rating_value': rating_value, 'author': author,
                    'job_title': job_title, 'description': description, 'keywords': keywords, 'categories': categories,
                    'cuisine': cuisine, 'ingredients': ingredients, 'steps': steps, 'recipe_yield': recipe_yield, 'time':time
                }



topic: method
method: 0 of 230 => https://www.seriouseats.com/recipes
method: 1 of 230 => https://www.seriouseats.com/recipes/2010/03/sous-vide-steaks-recipe.html
method: 2 of 230 => https://www.seriouseats.com/recipes/2016/04/sous-vide-pork-chops-recipe.html
method: 3 of 230 => https://www.seriouseats.com/recipes/2016/08/sous-vide-salmon-recipe.html
method: 4 of 230 => https://www.seriouseats.com/recipes/2016/07/sous-vide-pork-tenderloin-recipe.html
method: 5 of 230 => https://www.seriouseats.com/recipes/2015/09/sous-vide-pork-ribs-recipe-food-lab.html
method: 6 of 230 => https://www.seriouseats.com/recipes/2016/07/sous-vide-barbecue-pulled-pork-shoulder-recipe.html
method: 7 of 230 => https://www.seriouseats.com/recipes/2015/07/sous-vide-chicken-breast-recipe.html
method: 8 of 230 => https://www.seriouseats.com/recipes/2016/08/sous-vide-barbecue-smoked-bbq-brisket-texas-recipe.html


KeyboardInterrupt: 

In [15]:
collection = mongoDb['method']
se_method_links_dict = collection.find_one({}, {'_id': False})

In [14]:
collection = mongoDb['cuisine']
se_cuisine_links_dict = collection.find_one({}, {'_id': False})

In [16]:
se_recipe_links_dict = {'method': se_method_links_dict, 'cuisine': se_cuisine_links_dict}

In [ ]:
pprint(se_recipe_links_dict)

In [42]:
collection = mongoDb.se_recipes
collection.insert_one(se_recipe_links_dict)

In [229]:
pprint(se_recipe_links_dict['cuisine'][4]['cuisine_url(s)'])
# pprint(len(se_recipe_links_dict['cuisine'][4]['cuisine_url(s)']))

['https://www.seriouseats.com/recipes/topics/cuisine/caribbean']


In [282]:
test = {}

In [ ]:
se_recipe_links_dict.pop('_id')
pprint(se_recipe_links_dict)

In [244]:
for topic, _list in test.items():
    for _dict in _list[4:5]:
        for recipe in _dict['recipes']:
            print(recipe['recipe_title'])



          Recipes

Cuban-Style Roast Pork Shoulder With Mojo
An easy, pull-apart-tender, extra-porky roast.


Cuban Sandwiches
The king of ham and cheese sandwiches.


Jamaican Beef Stew With Rice


Curried Jamaican Beef Patties


Cuban Picadillo
Piccadillo is classic Cuban comfort food, made from ground meat mixed with a flavorful combination of olives, capers, and raisins. 


Ropa Vieja
Shredded and juicy meat bathed in a flavorful sauce.


Grilled Skirt Steak With Mojo Marinade
A garlicky, citrusy marinade makes this buttery steak pop.


Jamaican Chicken Curry


Spicy Grilled Jerk Chicken
To make spicy Jamaican-style jerk chicken, marinate and brine the bird, then smoke it low and slow on the grill. 


Jamaican Goat Curry
There are very few Jamaican dishes more popular or delicious than goat curry.


Faux Cubano Grilled Cheese
An easier spin on the Cubano.


Slow-Cooker Ropa Vieja With Black Beans and Rice
This Cuban dish of shredded stewed beef flavored with a vinegary tomato and

In [137]:
for anchor in recipe_anchors[::2]:
    print(anchor.text.strip())
    print('============')
    print(f"href: {anchor['href']}")
    print('======')

Recipes
href: https://www.seriouseats.com/recipes
Creamy Irish-Style Oatmeal With Brown Sugar
A morning indulgence that's creamy and lightly sweetened with brown sugar.
href: https://www.seriouseats.com/recipes/2019/02/creamy-irish-style-oatmeal-with-brown-sugar.html
New York-Style Pizza Sauce
Youse want a New Yawk Slice? Youse came to da right place.
href: https://www.seriouseats.com/recipes/2010/10/new-york-style-pizza-sauce.html
3-Ingredient Stovetop Macaroni and Cheese
So easy, it's practically done before you start.
href: https://www.seriouseats.com/recipes/2017/01/3-ingredient-stovetop-mac-and-cheese-recipe.html
The Best Chili Ever
This chili brings together a rich, complex chili flavor; a robust beefy character; and creamy, tender beans that stay intact.
href: https://www.seriouseats.com/recipes/2010/01/the-best-chili-recipe.html
Easy No-Bake Cheesecake
Simple, low-stress, and eminently satisfying.
href: https://www.seriouseats.com/recipes/2017/07/no-bake-cheesecake-recipe.html


[]

In [6]:

html = driver.page_source
se_soup = bs(html, 'html.parser')


In [7]:
script_divs = se_soup.find_all('script', {'type': 'application/ld+json'})

In [10]:
for div in script_divs:
    print(div.text)
    print('======')



  {
    "@context": "http://schema.org",
    "@type": "Organization",
    "name": "Serious Eats",
    "url": "https://www.seriouseats.com",
    "logo": {
      "@type": "ImageObject",
      "url": "https://static.seriouseats.com/1/braestar/live/img/logo-color-240x190.png"
    },
    "sameAs" : [ 
      "https://www.facebook.com/seriouseats",
      "https://twitter.com/seriouseats",
      "https://plus.google.com/+seriouseats/posts",
      "https://www.pinterest.com/seriouseats/",
      "https://www.instagram.com/seriouseats/",
      "https://www.youtube.com/user/SeriousEats"
    ] 
  }
  

    {
      "@context": "http://schema.org",
      "@type": "BreadcrumbList",
      "itemListElement":
      [  
          {
            "@type": "ListItem",
            "position": 1,
            "item":
            {
              "@id": "https://www.seriouseats.com/recipes",
              "name": "Recipes",
              "image" : "https://static.seriouseats.com/1/braestar/live/img/og-image-citru

In [29]:

j = json.loads(script_divs[2].text)
pprint(j)

{'@context': 'http://schema.org',
 '@type': 'BreadcrumbList',
 'itemListElement': [{'@type': 'ListItem',
                      'item': {'@id': 'https://www.seriouseats.com/recipes',
                               'image': 'https://static.seriouseats.com/1/braestar/live/img/og-image-citrus.jpg',
                               'name': 'Recipes'},
                      'position': 1},
                     {'@type': 'ListItem',
                      'item': {'@id': 'https://www.seriouseats.com/recipes/topics/ingredient',
                               'image': 'https://static.seriouseats.com/1/braestar/live/img/og-image-citrus.jpg',
                               'name': 'Ingredient'},
                      'position': 2},
                     {'@type': 'ListItem',
                      'item': {'@id': 'https://www.seriouseats.com/recipes/topics/ingredient/eggs',
                               'image': 'https://static.seriouseats.com/1/braestar/live/img/og-image-citrus.jpg',
              

267

In [36]:
tran_links = []
for tran in trans[1:267]:
    try: 
        raw = tran.find('a')['href'][2:]
#         print(raw)

        if raw[0:2] == '..':
            raw.replace('..', '')
            url = 'http://www.goodeatsfanpage.com' + raw
        else:
            url = 'http://www.goodeatsfanpage.com' + raw
        tran_links.append(url)
    except:
        pass